In [218]:
from google.colab import drive
drive.mount('/content/drive2',force_remount=True)

Mounted at /content/drive2


In [219]:
# read gazeteers
organizations = {}
locations = {}
persons = {}
organizational_gazeteer = open("/content/drive2/MyDrive/NLP/organizational_gazeteer.txt","r")
location_gazeteer = open("/content/drive2/MyDrive/NLP/location_gazeteer.txt","r")
person_gazeteer = open("/content/drive2/MyDrive/NLP/person_gazeteer.txt","r")

In [220]:
# create organization lists
for line in organizational_gazeteer:
  line = line.strip()
  organizations[line] = 1
# create location lists
for line in location_gazeteer:
  line = line.strip()
  locations[line] = 1

# create person lists
for line in person_gazeteer:
  line = line.strip()
  persons[line] = 1

In [221]:
# install libraries
!pip install sklearn_crfsuite
!pip install eli5

In [222]:
import sklearn_crfsuite
import nltk
import re
import eli5
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.model_selection import cross_validate
from sklearn.metrics import plot_confusion_matrix
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [347]:
all_sents = []
count = 0
with open("/content/drive2/MyDrive/NLP/NE.txt","r") as ner:
  for line in ner: # reading the file line by line
    train_sent = []
    count = 0
    #line = line.replace(" '","'")
    #line = line.replace("' ","'")
    #line = line.replace("  "," ")
    ner_type = ""
    b_enamex = 0
    start = 0
    end = 0
    for word in line.split():
      #print(word)
      if word == "<b_enamex": # this means we will encounter a word with tags other than 'O'
        b_enamex = 1
        start = 1
      elif "e_enamex" in word:
        b_enamex = 0
        if "TYPE" in word:
          match = re.findall('="([A-ZÇĞİÖŞÜ]*[a-zçğıöşü]*)"',word)
          if len(match):
            match = match[0]
            #ner_type = match
            if start:
                ner_type = "B-"
                start = 0
            else:
              ner_type = "I-"
            if match == "ORGANIZATION":
              ner_type += "ORG"
            if match == "LOCATION":
              ner_type += "LOC"
            if match == "PERSON":
              ner_type += "PER"
            
            word = re.findall('>([A-ZÇĞİÖŞÜ]*[a-zçğıöşü]*)\s*',word)
            word =word[0]
            #print("-----",word,ner_type)
            real_ner = "B" # it was 'U'
            real_ner += ner_type[1:]
            ner_type = real_ner
            train_sent.append([word,ner_type])
            count += 1
          #print("ENAMEX TYPE:",match)
        else:
          if start:
            ner_type = "B-"
            start = 0
          else:
            ner_type = "I-"
          if match == "ORGANIZATION":
            ner_type += "ORG"
          if match == "LOCATION":
            ner_type += "LOC"
          if match == "PERSON":
            ner_type += "PER"
          word = re.findall('>?([A-ZÇĞİÖŞÜ]*[a-zçğıöşü]*)<e_enamex>',word)
          if len(word):
            word = word[0]
            train_sent.append([word,ner_type])
            count += 1
      else:
        if b_enamex:
          if "TYPE" in word:
            match = re.findall('="([A-ZÇĞİÖŞÜ]*[a-zçğıöşü]*)"',word)
            if len(match):
              match = match[0]
              #print("************start",start)
              if start:
                ner_type = "B-"
                start = 0
              else:
                ner_type = "I-"
              if match == "ORGANIZATION":
                ner_type += "ORG"
              if match == "LOCATION":
                ner_type += "LOC"
              if match == "PERSON":
                ner_type += "PER"

              #print("TYPE:",ner_type)
            word = re.findall('>([A-ZÇĞİÖŞÜ]*[a-zçğıöşü]*)\s*',word)
            word =word[0]
            #print("-----",word,ner_type)
            train_sent.append([word,ner_type])
            count += 1
          else:
            if start:
              ner_type = "B-"
              start = 0
            else:
              ner_type = "I-"
            if match == "ORGANIZATION":
                ner_type += "ORG"
            if match == "LOCATION":
              ner_type += "LOC"
            if match == "PERSON":
              ner_type += "PER"
            if word != "<b_enamex":
              #print("Aradakiler",word,ner_type)
              train_sent.append([word,ner_type])
              count += 1
        else:
          # if word.startswith("'"):
          #  if count >= 1:
          #    train_sent[count-1][0] += word
          #else:
            train_sent.append([word,'O']) # they were indented
            count += 1 # they were indented
    
        
    all_sents.append(train_sent)
      
    #if count == 12:
      #break    

In [348]:
analysis_list = []
analysis_words = []
analysis_pairs  = []
prev_line = 1
sentences = []
with open("/content/drive2/MyDrive/NLP/NE.ma.txt","r") as ner_ma: # read the morphological nalaysis file
  count = 0
  for line in ner_ma:
    line = line.split()
    analysis_list.append([line[0],line[1],line[2]])
    analysis_words.append(line[1])
    if (int(line[0]) > prev_line):
      analysis_pairs.append(sentences)
      sentences = []
      sentences.append([line[1],line[2]])
      prev_line = int(line[0])
    else:
      sentences.append([line[1],line[2]])
  
  analysis_pairs.append(sentences)

In [349]:
X_train_all = []
y_train_all = []
for i in range(len(analysis_pairs)):
  labels = []
  sentence = []
  count = 0
  for j in analysis_pairs[i]:
    word = j[0]
    exist = 0
    for pair in all_sents[i]:
      if word == pair[0]:
        exist = 1
        labels.append(pair[1]) 
        #j.append(pair[1])
        break
    if not exist:
      labels.append('O')
    # add that word to the dictionary pair
    feature_dict = {}
    analysis = j[1]
    first_index = analysis.find("+") # get stem of the word
    stem = analysis[:first_index]
    feature_dict['Stem'] = stem
    second_index = analysis.find("+",first_index + 1)
    POS = analysis[first_index +1:second_index]
    feature_dict['POS'] = POS # get POS of the word
    PROP = True if "+Prop" in analysis else False # check whether the analysis of the word includes '+Prop'
    feature_dict['PROP'] = PROP
    last_index = analysis.rfind("+")
    NCS = analysis[last_index+1:]
    #print(NCS)
    if NCS == "Nom" or NCS == "Acc" or NCS == "Dat" or NCS == "Abl" or NCS == "Loc" or NCS == "Gen" or NCS == "Ins" or NCS == "Equ":
      feature_dict['NCS'] = NCS
    else:
      feature_dict['NCS'] = False # non nominal

    INF = analysis[second_index+1:]
    feature_dict['INF'] = INF
    feature_dict['SS'] = True if count == 0 else False
    feature_dict['OCS'] = True if word[0].isupper() else False
    feature_dict['word.lower()'] = word.lower()
    feature_dict['Person'] = False
    feature_dict['Organization'] = False
    feature_dict['Location'] = False
    if word in persons.keys():    # checking if the word exists in my gazetters
      feature_dict['Person'] = True
    if word in locations.keys():
      feature_dict['Location'] = True
    if word in organizations.keys():
      feature_dict['Organization'] = True
    feature_dict['word.isupper()'] = word.isupper()
    feature_dict['word.istitle()'] = word.istitle() # check whether onlt the first letter of the word is capitalized or not 
    feature_dict['word.isdigit()'] = word.isdigit()
    feature_dict['word[-2:]'] = word[-2:] # last 2 letters
    feature_dict['word[-3:]'] = word[-3:] # last 3 letters
    sentence.append(feature_dict)
    count += 1
  X_train_all.append(sentence)
  
  y_train_all.append(labels)
  #j.append('O')

In [350]:
# prepare data for cross validation described in the document
new_X_train_all = []
new_y_train_all = []
for i in range(5):
  count = i
  while (count + 5 < len(X_train_all)):
    new_X_train_all.append(X_train_all[count])
    new_y_train_all.append(y_train_all[count])
    count += 5

In [351]:
import numpy as np  # get the numpy arrays to use them in the cross validation
X = np.array(new_X_train_all)
y = np.array(new_y_train_all)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [352]:
from sklearn.model_selection import cross_val_score
# build the models
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False,
)


In [353]:
from sklearn.model_selection import KFold

results = {'B-LOC':[0,0,0],'B-ORG':[0,0,0],'B-PER':[0,0,0],'I-LOC':[0,0,0],'I-ORG':[0,0,0],'I-PER':[0,0,0]} # store the averages of f1-score, recall and precision
count = 1
for train_index, test_index in kf.split(X,y):
  #print(test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  crf.fit(X_train,y_train)
  y_pred = crf.predict(X_test)
  labels = list(crf.classes_)
  labels.remove('O')  # remove the 'O' tag
  #labels
  print("Fold number",count)
  count += 1
  print(metrics.flat_classification_report(y_test, y_pred, labels = labels))
  report_dict = metrics.flat_classification_report(y_test, y_pred, labels = labels,output_dict = True)
  for key in results.keys():
    results[key][0] += report_dict[key]['f1-score']
    results[key][1] += report_dict[key]['precision']
    results[key][2] += report_dict[key]['recall']
  
  print("*************************************")

for key in results.keys(): # compute the averages
  results[key][0] /= 5
  results[key][1] /= 5
  results[key][2] /= 5

#print average dict
print(results)
  #print("%s %s" % (train, test))

Fold number 1
              precision    recall  f1-score   support

       B-LOC       0.93      0.84      0.88       489
       B-PER       0.91      0.89      0.90       863
       I-PER       0.89      0.87      0.88       333
       B-ORG       0.87      0.79      0.83       494
       I-ORG       0.80      0.77      0.78       331
       I-LOC       0.81      0.58      0.68        67

   micro avg       0.89      0.84      0.86      2577
   macro avg       0.87      0.79      0.83      2577
weighted avg       0.89      0.84      0.86      2577

*************************************
Fold number 2
              precision    recall  f1-score   support

       B-ORG       0.83      0.76      0.79       443
       I-ORG       0.75      0.70      0.72       310
       B-LOC       0.89      0.89      0.89       464
       B-PER       0.93      0.87      0.90       886
       I-PER       0.89      0.87      0.88       385
       I-LOC       0.72      0.60      0.65        52

   micro av

In [120]:
# %%time
# import scipy
# from sklearn.metrics import fbeta_score, make_scorer
# from sklearn.model_selection import RandomizedSearchCV
# # define fixed parameters and parameters to search
# crf = sklearn_crfsuite.CRF(
#     algorithm='lbfgs',
#     max_iterations=100,
#     all_possible_transitions=True
# )
# params_space = {
#     'c1': scipy.stats.expon(scale=0.5),
#     'c2': scipy.stats.expon(scale=0.05),
# }

# # use the same metric for evaluation
# f1_scorer = make_scorer(metrics.flat_f1_score,
#                         average='weighted', labels=labels)

# # search
# rs = RandomizedSearchCV(crf, params_space,
#                         cv=3,
#                         verbose=1,
#                         n_jobs=-1,
#                         n_iter=50,
#                         scoring=f1_scorer)
# rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 29.8min finished


CPU times: user 23min 47s, sys: 11.6 s, total: 23min 58s
Wall time: 29min 57s
